In [1]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub
from langchain_community.tools.tavily_search import TavilySearchResults

In [2]:
load_dotenv()

api_key = os.getenv('OPENAI_API_KEY')
tavily_api_ley = os.getenv('TAVILY_API_KEY')

if not api_key or not tavily_api_ley:
    print("api key is not loaded")
else:
    print("api key is loaded")

api key is loaded


In [3]:
def get_profile_info(name: str):
    search = TavilySearchResults()
    res = search.run(f"{name}")
    return res

In [4]:
def linkedin_lookup_agent(name: str) -> str:
    llm = ChatOpenAI(
        model_name="gpt-4o-mini",
        temperature=0
    )

    template = """
        given the full name {name_of_person} I want you to get it me a link to their Linkedin profile page. 
        Your answer should contain only a url
    """

    prompt_template = PromptTemplate(
        template=template,
        input_variables=["name_of_person"]
    )

    tools_of_agent = [
        Tool(
            name="Crawl Google for Linkedin profile page",
            func=get_profile_info,
            description="It is useful when you need to get the Linkedin page url"
        )
    ]

    react_prompt = hub.pull("hwchase17/react")
    agent = create_react_agent(
        llm=llm,
        tools=tools_of_agent,
        prompt=react_prompt
    )
    agent_executor = AgentExecutor(
        agent=agent,
        tools=tools_of_agent,
        verbose=True
    )

    result = agent_executor.invoke(
        input={"input": prompt_template.format_prompt(name_of_person=name)}
    )

    return result["output"]

In [5]:
def get_info_of_a_person(name: str):
    linkedin_url = linkedin_lookup_agent(name=name)

    summary_template = """
        given the information {information} about a person I want you to provide, 
        if you can't access external sites, added the widely known information:
        1. A short summary
        2. Two interesting facts about them
    """

    summary_prompt_template = PromptTemplate(
        input_variables=["information"],
        template=summary_template
    )
    
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    chain = summary_prompt_template | llm
    res = chain.invoke(input={"information": linkedin_url})
    print(res.content)
    

In [6]:
get_info_of_a_person("Bill Gates microsoft")

C:\Users\FahimAlSami\Documents\llm\venv\Lib\site-packages\langsmith\client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...
I need to find the LinkedIn profile page for Bill Gates, who is associated with Microsoft. To do this, I will perform a search on Google to locate his LinkedIn profile.  
Action: Crawl Google for Linkedin profile page  
[{'url': 'https://www.bfsinvest.com/bill-gates-and-the-story-of-microsoft', 'content': 'Bill Gates and the Story of Microsoft Bill Gates and the Story of Microsoft It is intended to help the reader grasp the ways in which Bill Gates and the company he cofounded, Microsoft, have changed the modern world. In 1975, Gates and Allen moved to Albuquerque and created a partnership called Micro-Soft (later renamed Microsoft) at the suggestion of Allen, as the company was working on microcomputer software instead of mainframes. Microsoft received only $430,000 from IBM — far less than IBM had expected to pay — but Gates and Allen gained the rights to sell QDOS, now called MS-DOS, to other manufacturing companies. In 1987, when Gates was 32